In [1]:
import pandas as pd
import os
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

In [2]:
with open('cnn_tokenizer.pickle', 'rb') as handle:
    cnn_tokenizer = pickle.load(handle)
handle.close()

In [3]:
with open('label_encoder.pickle', 'rb') as handle:
    le = pickle.load(handle)
handle.close()

In [8]:
cnn = load_model("cnn.h5")

In [4]:
with open('lstmx_tokenizer.pickle', 'rb') as handle:
    lstmx_tokenizer= pickle.load(handle)
handle.close()

In [5]:
with open('lstmy_tokenizer.pickle', 'rb') as handle:
    lstmy_tokenizer= pickle.load(handle)
handle.close()

In [7]:
bilstm= load_model("lstm.h5")

In [9]:
def predict(sentence):
  # Intent Classification
    sentence_sequence = cnn_tokenizer.texts_to_sequences([sentence])
    sentence_sequence_padded = pad_sequences(sentence_sequence, padding='post', maxlen=50)
    cnnpred= cnn.predict(sentence_sequence_padded)
    predicted_label = np.argmax(cnnpred)
    label=le.classes_[predicted_label]
  # Slots Filling
    label_list = list(lstmy_tokenizer.word_index.keys())
    index_list = list(lstmy_tokenizer.word_index.values())
    input_seq = lstmx_tokenizer.texts_to_sequences([sentence])
    input_features = pad_sequences(input_seq, maxlen = 50, padding = 'post')
    lstmpred = bilstm.predict(input_features)
    slots = [label_list[index_list.index(j)] for j in [np.argmax(x) for x in lstmpred [0][:]] if j in index_list]

  # Result
    print("Sentence:", sentence)
    print("Intent:", label)
    print("Slot:", slots)

In [10]:
predict("Tell me best Japanese restaurant and book me 3 seats")

1/1 [==============================] - 0s 340ms/step
Sentence: Tell me best Japanese restaurant and book me 3 seats
Intent: BookRestaurant
Slot: ['O', 'O', 'O', 'O', 'B-restaurant_type', 'O', 'B-restaurant_type', 'O', 'B-party_size_number', 'O']


In [12]:
converter_cnn = tf.lite.TFLiteConverter.from_keras_model(cnn)
tflite_model_cnn = converter_cnn.convert()
open("cnn.tflite", "wb").write(tflite_model_cnn)

5312020

In [13]:
bilstm = tf.keras.models.load_model("lstm.h5")

converter_bilstm = tf.lite.TFLiteConverter.from_keras_model(bilstm)
converter_bilstm.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS
]
converter_bilstm._experimental_lower_tensor_list_ops = False

tflite_model_bilstm = converter_bilstm.convert()
open("bilstm.tflite", "wb").write(tflite_model_bilstm)

2790296

In [15]:
interpreter_cnn = tf.lite.Interpreter(model_path="cnn.tflite")
interpreter_cnn.allocate_tensors()

interpreter_bilstm = tf.lite.Interpreter(model_path="bilstm.tflite")
interpreter_bilstm.allocate_tensors()

In [18]:
def predict_tflite(sentence):
    # Intent Classification (using CNN model)
    sentence_sequence = cnn_tokenizer.texts_to_sequences([sentence])
    sentence_sequence_padded = tf.keras.preprocessing.sequence.pad_sequences(sentence_sequence, padding='post', maxlen=50)

    input_details = interpreter_cnn.get_input_details()
    output_details = interpreter_cnn.get_output_details()

    interpreter_cnn.set_tensor(input_details[0]['index'], sentence_sequence_padded.astype(np.float32))
    interpreter_cnn.invoke()
    cnnpred = interpreter_cnn.get_tensor(output_details[0]['index'])
    predicted_label = np.argmax(cnnpred)
    label = le.classes_[predicted_label]

    # Slots Filling (using BiLSTM model)
    label_list = list(lstmy_tokenizer.word_index.keys())
    index_list = list(lstmy_tokenizer.word_index.values())
    input_seq = lstmx_tokenizer.texts_to_sequences([sentence])
    input_features = tf.keras.preprocessing.sequence.pad_sequences(input_seq, maxlen=50, padding='post')

    input_details = interpreter_bilstm.get_input_details()
    output_details = interpreter_bilstm.get_output_details()

    interpreter_bilstm.set_tensor(input_details[0]['index'], input_features.astype(np.float32))
    interpreter_bilstm.invoke()
    lstmpred = interpreter_bilstm.get_tensor(output_details[0]['index'])
    slots = [label_list[index_list.index(j)] for j in [np.argmax(x) for x in lstmpred[0][:]] if j in index_list]

    print("Sentence:", sentence)
    print("Intent:", label)
    print("Slot:", slots)

In [19]:
predict_tflite("Tell me best Japanese restaurant and book me 3 seats")

Sentence: Tell me best Japanese restaurant and book me 3 seats
Intent: BookRestaurant
Slot: ['O', 'O', 'O', 'O', 'B-restaurant_type', 'O', 'B-restaurant_type', 'O', 'B-party_size_number', 'O']
